In [1]:
import praw
import json
from datetime import datetime

In [2]:
def create_reddit_object(json_file = "data/reddit_config.json", json_key = "reddit"):
    with open(json_file) as f:
        data = json.load(f)
    user_values = data[json_key]
    reddit = praw.Reddit(client_id = user_values['client_id'],
                        client_secret=user_values['client_secret'],
                        user_agent=user_values['user_agent'],
                        username=user_values['username'],
                        password=user_values['password'],)
    return reddit

In [3]:
def write_json(path, data) :
    if path[-5:] != ".json":
        path = path +".json"
    with open(path, 'w', encoding = "utf-8") as outfile:
        outfile.write(json.dumps(data, ensure_ascii=False))

In [4]:
reddit = create_reddit_object()

In [5]:
# hot, new, controv, top, gilded
def scrap_subreddit(subreddit, limit):
    id = 0
    
    scrapList = []
    
    subReddit = reddit.subreddit(subreddit).hot(limit = limit)

    for submission in subReddit:
        scrapDict = {}
        commentList = []
        id += 1
        scrapDict["id"] = id
        scrapDict["title"] = submission.title
        date = datetime.fromtimestamp(submission.created_utc)
        scrapDict["date"] = str(date)
        for top_level_comment in submission.comments:
            commentList.append(top_level_comment.body.strip())

        scrapDict["comments"] = commentList
        if (id*100 // limit) % 20  == 0 :
            print(id*100 // limit,"%...")
        scrapList.append(scrapDict)
    print("Done!")
    return scrapList

In [6]:
data = scrap_subreddit(subreddit = "manhwa", limit = 5)

20 %...
40 %...
60 %...
80 %...
100 %...
Done!


In [7]:
write_json(path = "redditWebtoon.json", data = data)

In [8]:
print(data)

[{'id': 1, 'title': 'New Manhwa discord!', 'date': '2021-05-30 08:37:21', 'comments': ['Is there another season about the  ( I save female assassin) ?', 'Huh, was there an old one?']}, {'id': 2, 'title': 'PSA: Any post that has spoilers in the title or shows an image that is a spoiler without the spoiler tag will result in the post being taken down and result in a ban. Please report posts with spoilers so we can address them.', 'date': '2021-04-25 15:35:34', 'comments': []}, {'id': 3, 'title': 'Which ship is your favorite?', 'date': '2021-07-07 14:35:44', 'comments': ['TBATE ship actually has a story. Whereas in Solo, "You smell good, I wanna bang."', 'Sauce - Solo Leveling, Tbate, Unordinary', 'Unpopular opinion. None of them :v', "Unordinary's art looks so weird when compared to the other two 💀💀", "Tbae\n\n\n\nAlso i think tbae is much better than solo leveling, i don't know how good it is compared to other manhwas but for a power-fantasy it's pretty darn good.", 'Man as a TBATE nove

In [9]:
import pandas as pd
df = pd.read_json('redditWebtoon.json', orient = 'record', encoding = 'utf-8')

In [10]:
df[:2]

,id,title,date,comments
0,1,New Manhwa discord!,2021-05-30 08:37:21,[Is there another season about the ( I save f...
1,2,PSA: Any post that has spoilers in the title o...,2021-04-25 15:35:34,[]
